# Profitable app profiles for Google Play and the iOS App Store

The aim in this project is to find mobile app categories that are profitable in the Google Play and iOS App Store markets. The project should enable a company that builds iOS and Android mobile apps to make data-driven decisions with respect to the types of apps they provide.

The company only builds apps in English that are free to download and install, and their main source of revenue consists of in-app ads. This means that the number of users of their apps drives their revenue for any given app — the more users who see and engage with the ads, the better. The project's goal is to help the developers understand what kind of mobile apps tend to attract more users, and we need to recommend at least one app category.

To achieve this goal, we'll analyze publicly available data about mobile apps offered on Google Play and the iOS App Store. First we'll explore and clean up the data, and then we'll look into the most common and the most popular apps and offer some insights and suggestions.

## About the data

As of August 2024, there were about 2.3 million Android apps in Google Play Store, and roughly 2 million iOS apps available on the iOS App Store. According to [Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/), Apple and Google regularly remove low-quality content from their app stores, which might cause a fluctuation in the exact number of apps, but overall there's an increasing trend. 
![img](Images/number_image_edited_to_include_stores_Durshikshya_Aug_2024.png) Source: [Statista](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/) (photo via [Dursikshya](https://dursikshya.edu.np/details/mobile-app-architecture-and-how-to-start-building-one))

Collecting data for over 4 million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources by collecting new data ourselves, we can first try to see if we can find any relevant existing data at no cost. Luckily, these are two datasets that seem suitable for our purpose:

* [A dataset](https://www.kaggle.com/datasets/lava18/google-play-store-apps) containing data about 10,841 apps from Google Play. The dataset can be downloaded directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* [A dataset](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) containing data about 7,197 apps from the iOS App Store. The dataset can be downloaded directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

## Opening and exploring the datasets

We'll start by opening and exploring these two datasets. We'll keep the original dataset as tuples, since ideally we want to have them remain intact. This will allow us to come back to them later if we need to reference anything. We'll also create versions that exclude the header.

In [1]:
# Opening the Google Play dataset

from csv import reader
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android_dataset = tuple(read_file) # with header
android_header = android_dataset[0]
android_apps = android_dataset[1:] # without header

In [2]:
# Opening the iOS App Store dataset

from csv import reader
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios_dataset = tuple(read_file) # with header
ios_header = ios_dataset[0]
ios_apps = ios_dataset[1:] # without header

To make the datasets easier to explore, we'll first create a function named `explore_data()` that can be used repeatedly to print rows in a more readable way. We'll also add an option for the function to show the number of rows and columns for any dataset.

In [3]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row, '\n')

    if rows_and_columns:
        print('Number of rows: ', len(dataset))
        print('Number of columns: ', len(dataset[0])) # based on the first row

This function takes a slice of any dataset and prints the rows separated with line breaks. The `dataset` argument should be a tuple or a list of lists, and ideally it shouldn't have a header row, otherwise the function will end up printing and counting one more row compared to the actual length. The function prints the rows by iterating over the slice, rather than the dataset. The `start` and `end` arguments should be integers used to represent where the slice should start and end. Finally, the Boolean `rows_and_columns` is used to print the number of rows and columns for the entire dataset and is `False` by default (the function will count the header too, if it's present).

Now that the function is ready, let's dive in and explore the Google Play dataset first.

In [4]:
print(android_header)
print("\n")
explore_data(android_apps, 0, 3, True) # the dataset without the header

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

Number of rows:  10841
Number of columns:  13


The Google Play dataset has 10,841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are: 
* "App" (name)
* "Category" and "Genres" (they seem similar or related)
* "Reviews" (number of ratings)
* "Rating" (average value of the ratings)
* "Installs" (number of installs)
* "Price" and "Type" (Free or Paid)

Here are the explanations for all columns in the dataset.

| Column   Title | Explanation |
|---|:---|
| App | Application name |
| Category | Category the app belongs to |
| Rating | Overall user rating of the app |
| Reviews | Number of user reviews for the app |
| Size | Size of the app |
| Installs | Number of user downloads/installs for the app |
| Type | Paid or Free |
| Price | Price of the app |
| Content Rating | Age group the app is targeted at (Children / Mature 21+ / Adult) |
| Genres | An app can belong to multiple genres (apart from its main category) |
| Last Updated | Date when the app was last updated on Play Store |
| Current Ver | Current version of the app available on Play Store |
| Android Ver | Min required Android version |

Now let's take a look at the iOS App Store dataset.

In [5]:
print(ios_header)
print("\n")
explore_data(ios_apps, 0, 3, True) # the dataset without the header

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

Number of rows:  7197
Number of columns:  16


There are 7,197 iOS apps in this dataset, and the columns that seem interesting are: 
* "track_name", 
* "prime_genre", 
* "rating_count_tot" vs "rating_count_ver" (number of user ratings),
* "user_rating" vs "user_rating_ver" (average value of user ratings),
* "price" (along with "currency"). 

Not all column names are self-explanatory in this case; more details can be found in the [dataset documentation](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps). The columns that contain "ver" are about the current version of the app, while the others are for all versions. Below are the explanations for all columns for quick reference.

| Column   title | Explanation |
|---|:---|
| id | App ID |
| track_name | App Name |
| size_bytes | Size (in Bytes) |
| currency | Currency Type |
| price | Price amount |
| rating_count_tot | User Rating counts (for all version) |
| rating_count_ver | User Rating counts (for current version) |
| user_rating | Average User Rating value (for all version) |
| user_rating_ver | Average User Rating value (for current version) |
| ver | Latest version code |
| cont_rating | Content Rating |
| prime_genre | Primary Genre |
| sup_devices.num | Number of supporting devices |
| ipadSc_urls.num | Number of screenshots showed for display |
| lang.num | Number of supported languages |
| vpp_lic | Vpp Device Based Licensing Enabled |

## Mapping both datasets

You might notice that the two datasets don't have exactly the same columns, for example the iOS App Store dataset doesn't have an "Installs" column. Additionally, some columns from one dataset might correspond to 2 or more columns from the other dataset, eg "prime_genre". The table below maps the columns of interest from both datasets, showing all the possible combinations.

| Google Play | iOS App Store    |
|-------------|------------------|
| App (name)  | track_name       |
| Category    | prime_genre      |
| Genres      | prime_genre      |
| Reviews     | rating_count_tot |
| Reviews     | rating_count_ver |
| Rating      | user_rating      |
| Rating      | user_rating_ver  |
| Installs    | n/a              |
| Price       | price            |
| Type (Free or Paid) | n/a (can use price amount) |
| n/a         | currency         |

## Cleaning up the data
Before beginning the analysis, we need to clean up the data and make sure it's accurate, otherwise the results of the analysis would be wrong. The cleanup will include the following:
* Detect inaccurate data, and correct or remove it.
* Detect duplicate app entries, and remove the duplicates.

Furthermore, we need to prepare the data to fit the purpose of this specific project. The company we analyze the data for builds apps designed for an *English-speaking* audience, that are *free* to download and install. This means that we need to do the following modifications to the datasets:
* Filter out non-English apps like 爱奇艺PPS -《欢乐颂2》电视剧热播.
* Isolate the free from the paid apps.

### Deleting innacurate data

The Google Play dataset has a dedicated [discussion section](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion?sort=recent-comments), and there are [some discussions](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/164101) that outline an error for row 10,472 (excluding the header). Let's print this row and compare it against the header and against another row that is correct.

In [6]:
print(android_header, '\n')      # header
print(android_apps[10472], '\n') # incorrect row
print(android_apps[0])           # correct row

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The row 10,472 corresponds to the app "Life Made WI-Fi Touchscreen Photo Frame", and the category for this row has the value 1.9, while the rating is 19. This is clearly off, because the "Category" shouldn't be a number, and the maximum rating for a Google Play app is 5. This seems to have been caused by a missing value in the "Category" column, which shifts everything to the left, including the value of 1.9 which ended up in the wrong column (this is mentioned in the [previous discussion](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/164101)). Since the row has an error, we'll remove it using the `del` statement. 

Before we delete the row, we'll first create a new dataset as a list of lists from the original, given the original is a tuple and cannot be changed. Then we'll remove that row from the new dataset.

Be careful not to run this code more than once; the first time the duplicate will be replaced with the next row which will take the same index, but the next time we run the code we'd end up removing the correct row as well. The final number of rows for the Google Play dataset should be 10,840 (equal to 10,841 minus 1).

In [7]:
# please remember not to run this block of code more than once

android_apps_list = list(android_apps) # creating the new dataset

print(len(android_apps_list))

del(android_apps_list[10472])
print(len(android_apps_list))

10841
10840


### Removing duplicate entries for Google Play 

If we explore the Google Play dataset long enough or look at the [discussions](https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion) section, we'll notice there are duplicate entries for some apps. For instance, there are 4 entries for Instagram:

In [8]:
print(android_header, "\n")

for app in android_apps_list:
    name = app[0] # the first column in the row
    if name == "Instagram":
        print(app)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


To find out how many duplicate rows there are in total, we will create two lists, one for storing unique rows and the other for duplicate app names. Then we'll loop through the Google Play dataset and for each iteration:
* Save the app name to a variable called `name`.
* If `name` isn't in the `unique_apps` list, append it to the `unique_apps` list.
* Else (if `name` is already in the `unique_apps` list), append it to the `duplicate_apps` list.


In [9]:
unique_apps = []
duplicate_apps = []

for app in android_apps_list:
    name = app[0]
    if name not in unique_apps:
        unique_apps.append(name)
    else:
        duplicate_apps.append(name)

print("Number of unique apps: ", len(unique_apps))
print("Number of duplicate apps: ", len(duplicate_apps), "\n")
print("Examples of duplicate apps:", "\n", duplicate_apps[:10])

Number of unique apps:  9659
Number of duplicate apps:  1181 

Examples of duplicate apps: 
 ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


In total, there are 1,181 duplicate rows in the Google Play dataset.

When we analyze the data, we shouldn't count the apps more than once, which means we need to remove the duplicate entries and keep only one row per app. We could just remove the duplicate rows randomly, but there's probably a better way.

If we examine the rows we printed for the Instagram app above, we'll notice that they differ by the number of reviews, which is at the 4th position in each row (index 3). The different number of reviews show that the data was collected at different times.

In [10]:
print(android_header[3], "for Instagram")

for app in android_apps_list:
    name = app[0]
    if name == "Instagram":
        print(app[3])

Reviews for Instagram
66577313
66577446
66577313
66509917


We can build on this to create a criterion for removing the duplicates. The higher the number of reviews is, the more recent the data should be and the more relevant the ratings. Hence, we'll keep the row with the highest number of reviews, and remove the other duplicate rows.

For Google Play, after removing the 1,181 duplicate rows, we should be left with 9,659 rows.

In [11]:
print("Total length:", len(android_apps_list))
print("Expected length without duplicates:", len(android_apps_list) - 1181)

Total length: 10840
Expected length without duplicates: 9659


To remove the duplicates, we will do the following:

* Create a dictionary, where each dictionary key is a unique app name, and the corresponding dictionary value is the highest number of reviews for that app.
* Use that dictionary to create a new dataset, which will have only one unique row per app, and for each app, we'll only select the row with the highest number of reviews.

Let's start by building the dictionary, called `reviews_max`. First, we loop through the Google Play dataset and assign the name and the number of reviews (converted to `float`) to their respective variables. Then we'll use the `if elif` statement to create the dictionary.

In [12]:
reviews_max = {}

for app in android_apps_list:
    name = app[0] # the first column in the row
    n_reviews = float(app[3])

    if name not in reviews_max:
        reviews_max[name] = n_reviews
    elif name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews

print(len(reviews_max))

9659


We can see that the length of the new dictionary is equal to the expected, aka 9,659 unique apps.

Basically, if `name` does not exist as a key in the `reviews_max` dictionary, we create a new key-value pair in the dictionary where the key is the app name and the value is the number of reviews. If `name` already exists as a key in the `reviews_max` dictionary **and** its existing value (the number of reviews) is smaller than the current iteration of `n_reviews`, we update the smaller number with the current `n_reviews` for that key-value pair in the dictionary. We need to make sure not to use an `else` clause here, or the number of reviews will be updated even when `reviews_max[name] < n_reviews` evaluates to `False` (aka when `name` exists in the dictionary **and** the existing value is bigger or equal to the current iteration of `n_reviews`).

Onto the second step, we'll use the information stored in the `reviews_max` dictionary to create a new dataset where we keep only one row per app, which will be the one with the highest number of reviews. This way, we'll have the duplicates removed.

We'll create 2 empty lists: `android_apps_cleaned`, which will be a list of lists and store the new cleaned up dataset, and `already_added`, which will just store the app names and help us keep track of apps that we already added. We again loop through the Google Play dataset, and for every iteration:
* We isolate the name of the app and the number of reviews stored as variables.
* We add the entire current row (`app`) to the `android_apps_cleaned` list, and the `name` of the app to the `already_added` list, if:
    * The number of reviews (`n_reviews`) of the current app is the same as the maximum number of reviews for that app as stated in the `reviews_max` dictionary (the number itself can be found in the dictionary); and
    * The name of the app is not already in the `already_added` list. We need to add this supplementary condition to account for those cases where the highest number of reviews for an app is the same for more than one rows (for example, the Box app has 3 rows and the number of reviews is the same). In other words, if we check just for `n_reviews == reviews_max[name]`, we'd still have duplicate rows for some of the apps.

Finally, we 'll quickly explore the new `android_apps_cleaned` dataset to confirm that the number of rows is 9,659 as expected.

In [13]:
android_apps_cleaned = []
already_added = []

for app in android_apps_list:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_apps_cleaned.append(app)
        already_added.append(name)

print(android_header)
print("\n")
explore_data(android_apps_cleaned, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

Number of rows:  9659
Number of columns:  13


The code below confirms that the rows for the Box app indeed have the same number of reviews. 

In [14]:
print(android_header[3], "for Box")

for app in android_apps_list:
    name = app[0]
    if name == "Box":
        print(app[3])

Reviews for Box
159872
159872
159872


### Removing duplicate entries for iOS App Store

We also need to investigate for any duplicates in the iOS dataset. First let's explore the data, just as a reminder to see which columns are included and what's the total number of rows.

In [15]:
print(ios_header)
print("\n")
explore_data(ios_apps, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

Number of rows:  7197
Number of columns:  16


There's an ID column, which might imply all rows being unique, but let's double-check just in case by using the app names. We'll first create a new dataset that's a list of lists from the original like we did for Google Play, since we want to make changes to it later.

In [16]:
ios_apps_list = list(ios_apps)

unique_apps_ios = []
duplicate_apps_ios = []

for app in ios_apps_list:
    name = app[1]
    if name not in unique_apps_ios:
        unique_apps_ios.append(name)
    else:
        duplicate_apps_ios.append(name)

print("Number of unique apps: ", len(unique_apps_ios))
print("Number of duplicate apps: ", len(duplicate_apps_ios), "\n")
print("Examples of duplicate apps:", "\n", duplicate_apps_ios)

Number of unique apps:  7195
Number of duplicate apps:  2 

Examples of duplicate apps: 
 ['Mannequin Challenge', 'VR Roller Coaster']


There are duplicates only for 2 apps. Let's see if there are any differences between the rows for each. We'll create a function just for this purpose, in order to avoid repetition of code. At the same time, we'll create a list of lists with the duplicate rows and include the index for each row, which we'll come back to later. 

In [17]:
# please remember not to run this code of block more than once
# otherwise the indexes will get appended multiple times

ios_duplicate_rows = []
ios_duplicate_rows.append(ios_header) # add the header row
ios_duplicate_rows[0].append("index") # add an index column to the header

def duplicates_ios(app_name):
    for app in ios_apps_list:
        name = app[1] # the second column in the row
        if name == app_name:
            index = ios_apps_list.index(app)
            app.append(index)
            ios_duplicate_rows.append(app)
            print(app)

print(ios_duplicate_rows[0]) # same as print(ios_header), plus the index
print("\n")
            
duplicates_ios("Mannequin Challenge")
print("\n")
duplicates_ios("VR Roller Coaster")

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic', 'index']


['1173990889', 'Mannequin Challenge', '109705216', 'USD', '0.0', '668', '87', '3.0', '3.0', '1.4', '9+', 'Games', '37', '4', '1', '1', 2948]
['1178454060', 'Mannequin Challenge', '59572224', 'USD', '0.0', '105', '58', '4.0', '4.5', '1.0.1', '4+', 'Games', '38', '5', '1', '1', 4463]


['952877179', 'VR Roller Coaster', '169523200', 'USD', '0.0', '107', '102', '3.5', '3.5', '2.0.0', '4+', 'Games', '37', '5', '1', '1', 4442]
['1089824278', 'VR Roller Coaster', '240964608', 'USD', '0.0', '67', '44', '3.5', '4.0', '0.81', '4+', 'Games', '38', '0', '1', '1', 4831]


For both apps, we can see differences in multiple columns, but the most notable are `rating_count_tot` (number of user ratings for all versions) and `ver` (latest version). In other words, the duplicates are just different versions of the same app. The newer version has a higher number of user ratings, which makes sense.

In [18]:
# print(ios_duplicate_rows)

for row in ios_duplicate_rows:
    index = row[16]
    name = row[1]
    version = row[9]
    n_ratings = row[5]
    print(index, name, version,  n_ratings)

index track_name ver rating_count_tot
2948 Mannequin Challenge 1.4 668
4463 Mannequin Challenge 1.0.1 105
4442 VR Roller Coaster 2.0.0 107
4831 VR Roller Coaster 0.81 67


| index | track_name          | ver   | rating_count_tot | 
| ----- | :------------------ | :---- |:---------------- | 
| 2948  | Mannequin Challenge | 1.4   |              668 |
| 4463  | Mannequin Challenge | 1.0.1 |              105 |
| 4442  | VR Roller Coaster   | 2.0.0 |              107 |
| 4831  | VR Roller Coaster   | 0.81  |               67 | 

Now that we have the index for each duplicate row, we can remove them directly rather than having to write more complicated code and create dictionaries. To keep things consistent, we'll keep the rows with the higher total number of user ratings (and the latest version). Alternatively, we could keep the duplicates given they contain different ratings and other information for the different versions, but since the newest versions have a higher total number of user ratings and the others are outdated, we'll keep the rows only for the newest version of the respective app.

We'll remove the duplicates by simply using the `del` statement. But we need to be careful; first we need to remove the duplicate row for the app that is positioned lower (4,831), because if we remove the duplicate at the top first (4,463), the index of the bottom duplicate row would change. In addition, we need to make sure to run this code only once, as the second time and after we'd delete the next rows that take the places of the removed duplicates.

Given the iOS dataset has a total of 7,197 rows, after removing the duplicates we should end up with 7,195 rows. We can create a new cleaned up dataset similarly to Google Play, but to keep things simple for demonstration purposes and reduce repetition of code, we'll just remove the duplicates from the list of lists itself (the original tuple dataset will still remain intact).

In [19]:
# please remember not to run this block of code more than once

print("Total length:", len(ios_apps_list))
print("Expected length without duplicates:", len(ios_apps_list) - 2)

del(ios_apps_list[4831])
del(ios_apps_list[4463])

print("Actual length without duplicates:", len(ios_apps_list))

Total length: 7197
Expected length without duplicates: 7195
Actual length without duplicates: 7195


Now that we removed the duplicates from both datasets, we can get a quick overview of the number of apps in the new vs the original datasets, and then proceed with the next step.

In [20]:
print("Google Play")
print("New:", len(android_apps_cleaned))
print("Old:", len(android_apps)) # including the error row

Google Play
New: 9659
Old: 10841


In [21]:
print("iOS")
print("New:", len(ios_apps_list))
print("Old:", len(ios_apps))

iOS
New: 7195
Old: 7197


### Filtering out non-English apps

Given the company uses English for the apps they develop, we should aim to analyze only the apps designed for an English-speaking audience. However, both datasets have non-English apps. These are some examples:

In [22]:
print(ios_apps_list[813][1])
print(ios_apps_list[6731][1])
print("\n")
print(android_apps_cleaned[4412][0])
print(android_apps_cleaned[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
激ムズ！和のひとふで書き！ 〜頭をつかう脳トレパズルゲーム〜


中国語 AQリスニング
لعبة تقدر تربح DZ


These are not relevant to our analysis, so we need to remove them. One way to go about this is to remove each app with a name containing symbols that aren't commonly used in English text, which usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (!, ?, ;, etc), and other symbols (+, /, *, etc).

The characters commonly used in English text are encoded using the ASCII standard (American Standard Code for Information Interchange), and each character has a corresponding number associated with it. For example, the corresponding number for character "a" is 97, character "A" is 65, and character "爱" is 29,233. With the ASCII standard, the numbers associated with the most commonly used English characters fall between 0 and 127.

We can use this range to build a function to help us detect whether the app name contains non-English characters. If a character has a corresponding number that is less than or equal to 127, it belongs to the set of common English characters. If the number is greater than 127, this might indicate that the name of the app is probably non-English, and we'll remove those rows. However, we need to decide how many characters have to fall out of the 0-127 range to consider the app name non-English; we'll come back to this later.

The app names are stored as strings, and we'll need to check the corresponding numbers for each individual character. In Python, we can use indexing to select an individual character from a string, and we can iterate on the string using a `for loop`.

In [23]:
string = "Maps"
print(string[0])
print(string[1])
print(string[2])
print(string[3])

M
a
p
s


In [24]:
for character in string: # "Maps"
    print(character)

M
a
p
s


Before we build the function, we'll first gather all the app names from both datasets into list of lists.

In [25]:
google_app_names = []

for app in android_apps_cleaned:
    name = app[0]
    google_app_names.append(name)

ios_app_names = []

for app in ios_apps_list:
    name = app[1]
    ios_app_names.append(name)
    
print(len(google_app_names))
print(google_app_names[:3])

print(len(ios_app_names))
print(ios_app_names[:3])

9659
['Photo Editor & Candy Camera & Grid & ScrapBook', 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'Sketch - Draw & Paint']
7195
['Facebook', 'Instagram', 'Clash of Clans']


Next, we'll build the detecting function using the built-in `ord()` function that returns the corresponding encoding number of each character. 

In [26]:
def is_english(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


The function seems to work, but it treats some English apps as non-English, like "Docs To Go™ Free Office Suite" and "Instachat 😜". This is because the app names use emojis and other symbols like ™, whose associated numbers fall outside the 0-127 range. Because of this, using the function in its current form will end up removing useful apps, since many English apps will be incorrectly labeled as non-English.

In [27]:
print(ord("™"))
print(ord("😜"))

8482
128540


To minimize the risk of removing valuable data, we'll only remove an app if its name has more than three non-ASCII characters. This means all English apps with up to three emojis or other special characters will still be labeled as English. This filter function might still allow for some incorrectly identified apps to pass through on both sides, but it likely won't be too many, and the function seems good enough for now, since we shouldn't spend too much time on optimization at this point in our analysis.

Let's update the `is_english` function we just created. If the input string has more than three characters that fall outside the ASCII range (0 - 127), the function should return False, otherwise it should return True.

In [28]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

print(is_english("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(is_english("Docs To Go™ Free Office Suite"))
print(is_english("Instachat 😜"))

False
True
True


Finally, we'll use the new, updated function to filter out non-English apps from both datasets. We loop through each dataset; if the function returns True, this means the app is English, and the whole row will be appended to the respective list of English apps for each dataset. If it returns False, this means the app is probably non-English and will get filtered out.

In [29]:
android_english = []
ios_english = []

for app in android_apps_cleaned:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios_apps_list:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

print(android_header, "\n")
explore_data(android_english, 0, 3, True)
print("\n")
print(ios_header, "\n")
explore_data(ios_english, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

Number of rows:  9614
Number of columns:  13


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic', 'index'] 

[

We can see that the number of English apps is 9,614 for Google Play (detected 45 non-English apps) and 6,181 for iOS (detected 1,014 non-English apps). The iOS dataset has significantly more non-English apps than the Google Play dataset, which might be insteresting to explore in a future project.

In [30]:
print("non-English iOS :", len(ios_apps_list) - len(ios_english))
print("non-English Android:", len(android_apps_cleaned) - len(android_english))

non-English iOS : 1014
non-English Android: 45


### Isolating the free apps from the paid

As we mentioned in the introduction, the company only builds apps that are free to download and install, and their main source of revenue comes from in-app ads. The datasets contain both free and non-free apps, so we'll need to isolate only the free apps for this analysis. This will be the last step in the data cleaning process. In the next section, we're going to start analyzing the data.

If we look at the column headers, we can identify which columns describe the pricing of the apps. In the Google Play dataset, we can use `"Price"` and `"Type"` (the latter returns Free or Paid; if we're going to use this column, we'll need to double-check if it identifies the free apps correctly). While in the iOS dataset, we can use `"price"` and `"currency"`. ***

The mapping table at the beginning showed us that the two datasets don't share exactly the same columns. For example, the Google Play dataset doesn't have a `"currency"` column. However, both have a `"price"` column, which is enough for the purposes of this analysis. In addition, we don't really need the `"currency"` column anyway; at the end of the day we're interested in identifying only the free apps, where the `"price"` should be equal to 0.

In [31]:
print("Google Play columns:")
print(android_header, "\n")

print("iOS columns:")
print(ios_header)

Google Play columns:
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

iOS columns:
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic', 'index']


| Dataset       | Column   | Index |
| ------------- | -------- | ----- |
| Google Play   | Price    | 7     |
| Google Play   | Type     | 6     |
| iOS App Store | price    | 4     |
| iOS App Store | currency | 3     |

We're going to use the `"Price"` columns for both datasets. Before we proceed, we should check if they're strings (eg. \$2.99 or "0" with quotation marks), so that we can properly identify them.

In [32]:
print(android_english[0][7])
print(type(android_english[0][7]))

print(ios_english[0][4])
print(type(ios_english[0][4]))

0
<class 'str'>
0.0
<class 'str'>


Given they're strings in both datasets, we shouldn't check using `int` or `float` in the conditional statements. To keep things simple, we can leave the prices as strings and test the equality to zero by using the correct price formatting; all prices with 0 should technically have the same formatting, ie. "0" or "0.0", based on the examples above. If we get an error in the conditional statements, then we can reconsider converting them to numbers.

In [33]:
android_english_free = [] 
ios_english_free = []

for app in android_english:
    price = app[7]
    if price == "0":
        android_english_free.append(app)
    
for app in ios_english:
    price = app[4]
    if price == "0.0":
        ios_english_free.append(app)

print(len(android_english_free))
print(len(ios_english_free))

8864
3220


We're left with 8,864 Google Play apps and 3,220 iOS apps, which should be sufficient for the purposes of this project. This is where we conclude the cleanup of the data, and next we'll proceed with the analysis.

## Most common apps by Genre

As mentioned in the introduction, the company's aim is to determine the kinds of apps that are likely to attract more users. This is because the apps are free and the company's revenue is predomintantly reliant on ads, which means the number of people who use the apps is a big factor.

To optimize resources, the strategy for validating an app idea is comprised of three steps:

1. Build a minimal android version of the app, and add it to Google Play.
2. If the app has a good response from users, the company then develops it further.
3. If the app is profitable after six months, they'll also build an iOS version of the app and add it to the iOS App Store.

Because the company's end goal is to make its way into both Google Play and the iOS App Store, we need to recommend app profiles that show potential for being profitable in both markets. For example, a productivity app that makes use of gamification might work well in both markets.

### Creating frequency tables

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll need to build frequency tables using a few columns in our datasets. By exploring the data or by refering back to the mapping table at the beginning, we identify the relevant columns useful for this purpose: `Genres` and `Category` from the Google Play dataset, and the `prime_genre` column from the iOS App Store dataset. We'll buld frequency tables for both datasets.

In [34]:
print(android_header, "\n")
explore_data(android_english_free, 0, 3, True)

print("\n")

print(ios_header, "\n")
explore_data(ios_english_free, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

Number of rows:  8864
Number of columns:  13


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic', 'index'] 

[

From here on out, we'll use the final, cleaned up datasets: `android_english_free` and `ios_english_free`. Below is a mini-mapped table with the relevant columns, with the corresponding index numbers in square brackets.

| Google Play    | iOS App Store    |
| -------------: | ---------------: |
| App (name) [0] | track_name [1]   |
| Category [1]   | prime_genre [11] |
| Genres [9]     | prime_genre [11] |

Let's first build a frequency table for the iOS dataset. We'll do this by creating a dictionary, where each genre is the key, and the number of times the genre shows up is the corresponding value.

In [35]:
ios_genres_basic = {}

for app in ios_english_free:
    genre = app[11]
    if genre in ios_genres_basic:
        ios_genres_basic[genre] += 1
    else:
        ios_genres_basic[genre] = 1

print(ios_genres_basic)

{'Social Networking': 106, 'Photo & Video': 160, 'Games': 1872, 'Music': 66, 'Reference': 18, 'Health & Fitness': 65, 'Weather': 28, 'Utilities': 81, 'Travel': 40, 'Shopping': 84, 'News': 43, 'Navigation': 6, 'Lifestyle': 51, 'Entertainment': 254, 'Food & Drink': 26, 'Sports': 69, 'Book': 14, 'Finance': 36, 'Education': 118, 'Productivity': 56, 'Business': 17, 'Catalogs': 4, 'Medical': 6}


| prime_genre | count |
| :---- | ----- |
| Social Networking | 106 |
| Photo & Video | 160 |
| Games | 1872 |
| Music | 66 |
| Reference | 18 |
| Health & Fitness | 65 |
| Weather | 28 |
| Utilities | 81 |
| Travel | 40 |
| Shopping | 84 |
| News | 43 |
| Navigation | 6 |
| Lifestyle | 51 |
| Entertainment | 254 |
| Food & Drink | 26 |
| Sports | 69 |
| Book | 14 |
| Finance | 36 |
| Education | 118 |
| Productivity | 56 |
| Business | 17 |
| Catalogs | 4 |
| Medical | 6 |
| ------- | ------- |
| **Total** | **3220** |

The iOS dataset has 23 genres. The sum of all frequency values is 3220, which is the same as the length of the `ios_english_free` dataset. This is how we can check if the code works properly.

When it comes to the Google Play dataset, we can build a dictionary with 1 key and 2 values for each, one count for `"Category"` and the other for `"Genres"`, but this might be more complicated or confusing. Instead, let's build a separate frequency table for each of the 2 columns. This will also help us find out how they're different, and see which one is more compact (has less groups) and matches better with the genres from the iOS dataset.

In [36]:
android_categories_basic = {}
android_genres_basic = {}

for app in android_english_free:
    category = app[1]
    if category in android_categories_basic:
        android_categories_basic[category] += 1
    else:
        android_categories_basic[category] = 1
    
    genre = app[9]
    if genre in android_genres_basic:
        android_genres_basic[genre] += 1
    else:
        android_genres_basic[genre] = 1

print("Categories \n", android_categories_basic, "\n")
print("Genres \n", android_genres_basic)

Categories 
 {'ART_AND_DESIGN': 57, 'AUTO_AND_VEHICLES': 82, 'BEAUTY': 53, 'BOOKS_AND_REFERENCE': 190, 'BUSINESS': 407, 'COMICS': 55, 'COMMUNICATION': 287, 'DATING': 165, 'EDUCATION': 103, 'ENTERTAINMENT': 85, 'EVENTS': 63, 'FINANCE': 328, 'FOOD_AND_DRINK': 110, 'HEALTH_AND_FITNESS': 273, 'HOUSE_AND_HOME': 73, 'LIBRARIES_AND_DEMO': 83, 'LIFESTYLE': 346, 'GAME': 862, 'FAMILY': 1676, 'MEDICAL': 313, 'SOCIAL': 236, 'SHOPPING': 199, 'PHOTOGRAPHY': 261, 'SPORTS': 301, 'TRAVEL_AND_LOCAL': 207, 'TOOLS': 750, 'PERSONALIZATION': 294, 'PRODUCTIVITY': 345, 'PARENTING': 58, 'WEATHER': 71, 'VIDEO_PLAYERS': 159, 'NEWS_AND_MAGAZINES': 248, 'MAPS_AND_NAVIGATION': 124} 

Genres 
 {'Art & Design': 53, 'Art & Design;Creativity': 6, 'Auto & Vehicles': 82, 'Beauty': 53, 'Books & Reference': 190, 'Business': 407, 'Comics': 54, 'Comics;Creativity': 1, 'Communication': 287, 'Dating': 165, 'Education': 474, 'Education;Creativity': 4, 'Education;Education': 30, 'Education;Pretend Play': 5, 'Education;Brain Game

| Category | count |
| :------- | -------- |
| Art_and_Design | 57 |
| Auto_and_Vehicles | 82 |
| Beauty | 53 |
| Books_and_Reference | 190 |
| Business | 407 |
| Comics | 55 |
| Communication | 287 |
| Dating | 165 |
| Education | 103 |
| Entertainment | 85 |
| Events | 63 |
| Finance | 328 |
| Food_and_Drink | 110 |
| Health_and_Fitness | 273 |
| House_and_Home | 73 |
| Libraries_and_Demo | 83 |
| Lifestyle | 346 |
| Game | 862 |
| Family | 1676 |
| Medical | 313 |
| Social | 236 |
| Shopping | 199 |
| Photography | 261 |
| Sports | 301 |
| Travel_and_Local | 207 |
| Tools | 750 |
| Personalization | 294 |
| Productivity | 345 |
| Parenting | 58 |
| Weather | 71 |
| Video_players | 159 |
| News_and_Magazines | 248 |
| Maps_and_Navigation | 124 |
| ------- | ------- |
| **Total** | **8864** |

The `Category` frequency table gives us 33 unique categories, which is much smaller than the number of unique `Genres` which is 114 (we skipped creating a markdown table for this to save space). Given that the number for `Category` is closer to the iOS number of `prime_genre` and that they look quite similar, we will proceed using the `Category` column for the Google Play dataset. We can compare the `Category` column with `Genres` at another time, but right now we're interested in the bigger picture. ***

Instead of generating the frequency tables manually, we can build two functions to help us with the analysis. The first will generate frequency tables and include an option to show the values as percentages, while the second will display the values in a descending order. Before building the functions, we'll create the frequency tables showing percentages, just to test out the code.

In [37]:
# Please run this block of code only once, otherwise it will recalculate

print("Total number of apps:", len(ios_english_free), "\n")
# print(ios_genres_basic)

for genre in ios_genres_basic:
    ios_genres_basic[genre] /= len(ios_english_free) 
    ios_genres_basic[genre] *= 100
    ios_genres_basic[genre] = round(ios_genres_basic[genre], 2)

print("Percentage table")
print(ios_genres_basic)

Total number of apps: 3220 

Percentage table
{'Social Networking': 3.29, 'Photo & Video': 4.97, 'Games': 58.14, 'Music': 2.05, 'Reference': 0.56, 'Health & Fitness': 2.02, 'Weather': 0.87, 'Utilities': 2.52, 'Travel': 1.24, 'Shopping': 2.61, 'News': 1.34, 'Navigation': 0.19, 'Lifestyle': 1.58, 'Entertainment': 7.89, 'Food & Drink': 0.81, 'Sports': 2.14, 'Book': 0.43, 'Finance': 1.12, 'Education': 3.66, 'Productivity': 1.74, 'Business': 0.53, 'Catalogs': 0.12, 'Medical': 0.19}


In [38]:
# Please run this block of code only once, otherwise it will recalculate

print("Total number of apps:", len(android_english_free), "\n")
# print(android_categories_basic)

for category in android_categories_basic:
    android_categories_basic[category] /= len(android_english_free) 
    android_categories_basic[category] *= 100
    android_categories_basic[category] = round(android_categories_basic[category], 2)

print("Percentage table")
print(android_categories_basic)

Total number of apps: 8864 

Percentage table
{'ART_AND_DESIGN': 0.64, 'AUTO_AND_VEHICLES': 0.93, 'BEAUTY': 0.6, 'BOOKS_AND_REFERENCE': 2.14, 'BUSINESS': 4.59, 'COMICS': 0.62, 'COMMUNICATION': 3.24, 'DATING': 1.86, 'EDUCATION': 1.16, 'ENTERTAINMENT': 0.96, 'EVENTS': 0.71, 'FINANCE': 3.7, 'FOOD_AND_DRINK': 1.24, 'HEALTH_AND_FITNESS': 3.08, 'HOUSE_AND_HOME': 0.82, 'LIBRARIES_AND_DEMO': 0.94, 'LIFESTYLE': 3.9, 'GAME': 9.72, 'FAMILY': 18.91, 'MEDICAL': 3.53, 'SOCIAL': 2.66, 'SHOPPING': 2.25, 'PHOTOGRAPHY': 2.94, 'SPORTS': 3.4, 'TRAVEL_AND_LOCAL': 2.34, 'TOOLS': 8.46, 'PERSONALIZATION': 3.32, 'PRODUCTIVITY': 3.89, 'PARENTING': 0.65, 'WEATHER': 0.8, 'VIDEO_PLAYERS': 1.79, 'NEWS_AND_MAGAZINES': 2.8, 'MAPS_AND_NAVIGATION': 1.4}


The blocks above show us the frequency tables for each dataset expressed in percentages. We can refer to them if we want to check whether the new function works properly.

We will build the first function in a way that allows us to choose between returning the count frequency table, or the percentages table. And we'll also include an parameter for rounding the percentages. By default, the `freq_table` function will return rounded percentages. The `dataset` argument is meant to be a list of lists, while the `index` should be used to choose the correct column from the respective dataset. This way, the function can be used for either dataset. Finally, the function will be used to create dictionaries that we can reuse later. 

In [39]:
def freq_table(dataset, index, percentages=True, rounded=True): 
    dictionary_count = {}
    dictionary_percentage = {}
    
    for app in dataset:
        field = app[index]
        if field in dictionary_count:
            dictionary_count[field] += 1
        else:
            dictionary_count[field] = 1

    for field in dictionary_count:
        dictionary_percentage[field] = (dictionary_count[field] / len(dataset)) * 100
        if rounded == True:
            dictionary_percentage[field] = round(dictionary_percentage[field], 2)
    
    if percentages == True:
        return dictionary_percentage
    else:
        return dictionary_count

ios_prime_genres = freq_table(ios_english_free, 11)
print("iOS prime genres: ")
print(ios_prime_genres)
print("\n")

android_category = freq_table(android_english_free, 1)
print("Google Play categories: ")
print(android_category)

iOS prime genres: 
{'Social Networking': 3.29, 'Photo & Video': 4.97, 'Games': 58.14, 'Music': 2.05, 'Reference': 0.56, 'Health & Fitness': 2.02, 'Weather': 0.87, 'Utilities': 2.52, 'Travel': 1.24, 'Shopping': 2.61, 'News': 1.34, 'Navigation': 0.19, 'Lifestyle': 1.58, 'Entertainment': 7.89, 'Food & Drink': 0.81, 'Sports': 2.14, 'Book': 0.43, 'Finance': 1.12, 'Education': 3.66, 'Productivity': 1.74, 'Business': 0.53, 'Catalogs': 0.12, 'Medical': 0.19}


Google Play categories: 
{'ART_AND_DESIGN': 0.64, 'AUTO_AND_VEHICLES': 0.93, 'BEAUTY': 0.6, 'BOOKS_AND_REFERENCE': 2.14, 'BUSINESS': 4.59, 'COMICS': 0.62, 'COMMUNICATION': 3.24, 'DATING': 1.86, 'EDUCATION': 1.16, 'ENTERTAINMENT': 0.96, 'EVENTS': 0.71, 'FINANCE': 3.7, 'FOOD_AND_DRINK': 1.24, 'HEALTH_AND_FITNESS': 3.08, 'HOUSE_AND_HOME': 0.82, 'LIBRARIES_AND_DEMO': 0.94, 'LIFESTYLE': 3.9, 'GAME': 9.72, 'FAMILY': 18.91, 'MEDICAL': 3.53, 'SOCIAL': 2.66, 'SHOPPING': 2.25, 'PHOTOGRAPHY': 2.94, 'SPORTS': 3.4, 'TRAVEL_AND_LOCAL': 2.34, 'TOOLS': 

### Ordering the frequency tables in descending order

Dictionaries don't have order, which makes it difficult to analyze the frequency tables. We're going to build a second function that can help us display the frequency table in a descending order. To do that, we'll need to make use of the built-in `sorted()` function. To make the order descending, we'll need to set the `reverse` parameter to True. Let's test this out.

In [40]:
print(sorted(ios_prime_genres, reverse=True))

['Weather', 'Utilities', 'Travel', 'Sports', 'Social Networking', 'Shopping', 'Reference', 'Productivity', 'Photo & Video', 'News', 'Navigation', 'Music', 'Medical', 'Lifestyle', 'Health & Fitness', 'Games', 'Food & Drink', 'Finance', 'Entertainment', 'Education', 'Catalogs', 'Business', 'Book']


In this case, only the names of the genres get returned, but not the frequencies. This is because for dictionaries, the `sorted()` function sorts by and returns the dictionary keys, not the values. We can circumvent this by using tuples and lists. By transforming the dictionary into a list of tuples or list or lists, we'll be able to sort by the values (the percentages or the count frequencies), by having each tuple/list containing the key-value pair, where the values comes first and the keys come second. Let's use a quick example to see how this function works with a list of tuples.

In [41]:
#dictionary
example_freq_table = {"Abc": 30, "Bcd": 75, "Cde": 15}

# lists of tuples
example_freq_table_as_tuple = [("Abc", 30), ("Bcd", 75), ("Cde", 15)]
example_freq_table_as_tuple2 = [(30, "Abc"), (75, "Bcd"), (15, "Cde")] 

print(sorted(example_freq_table_as_tuple, reverse=True))
print(sorted(example_freq_table_as_tuple2, reverse=True))

[('Cde', 15), ('Bcd', 75), ('Abc', 30)]
[(75, 'Bcd'), (30, 'Abc'), (15, 'Cde')]


The example above shows that the `sorted()` function basically takes whatever value comes first in each tuple, and sorts by those values (it works the same way with list of lists). This is why we need to transform the dictionaries in a way that the values come first. After the sorting, we can revert back the list of tuples/lists into a dictionary, or print the values as such.

Using this workaround, we'll write the second function, that takes in `dictionary` as parameter. For this parameter we'll use a dictionary that's already defined as a standalone variable, which we have created above. The function will transform the dictionary into a list of tuples, where the values come first and the keys second. Then it will sort in descending order following the values. Finally, it will print each pair of the sorted list of tuples, bringing back the key in first and the value in second place.

In [42]:
def display_sorted_table(dictionary):
    dictionary_as_tuples = [] # list of tuples

    for key in dictionary:
        tuple_pair = (dictionary[key], key)
        dictionary_as_tuples.append(tuple_pair)
    sorted_dictionary_as_tuples = sorted(dictionary_as_tuples, reverse=True)
    
    for pair in sorted_dictionary_as_tuples:
        print(pair[1], ":", pair[0]) # print key : value

### Analyzing the frequency tables

Now that the functions are ready, we'll use them to display the frequency tables for the genres in descending order. We'll first examine the `prime_genre` column from the iOS dataset. We'll also provide some graphs for visual assistance.

In [43]:
display_sorted_table(ios_prime_genres)

Games : 58.14
Entertainment : 7.89
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.52
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.34
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12


![img_prime_genre_iOS](Images/iOS_prime_genre.png)

We can see that among the free English apps in the iOS App Store, more than a half (58.14%) are games, making the distribution highly skewed. Entertainment apps are close to 8%, followed by photo & video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps.

According to the dataset, the general impression is that the iOS App Store (at least the part containing free English apps) is dominated by apps designed for fun (games, entertainment, photo and video, social networking, etc.), while apps with practical purposes (education, shopping, utilities, productivity, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they have the greatest number of users — the demand might not be the same as the offer. In other words, there might be numerous games offered, but the number of users for practical apps could still be greater.

Let's continue with the Google Play dataset, where we'll examine the `Category` and `Genres` columns. Note that we can quickly see the single most common value in each column from the [source URL](https://www.kaggle.com/datasets/lava18/google-play-store-apps), under "googleplaystore.csv" section in the "Column" tab, which can help confirm if the functions are working properly.

In [44]:
display_sorted_table(android_category)

FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6


![img_Category_Google_Play](Images/Google_Play_Category.png)

The landscape seems significantly different on Google Play: Family and Games dominate at the top with a combined 28.63%, but the next few categories indicate apps designed for practical purposes (tools, business, lifestyle, productivity, finance, medical, etc.). The distribution is more spread out unlike the iOS App Store.

If we investigate this further, we see that the "Family" category, which accounts for almost 19% of the apps, means mostly games for kids, as shown on the first photo below. The second photo shows the new, renamed version of the same category, called "Kids", which again is dominated by games, accompanied with some educational apps.

The limit of the Google Play dataset, as well as the iOS dataset, is that they are a few years old each at the time of making this project, and do not necessarily reflect the app stores at their current state. However, since this notebook is only for demonstration purposes, we'll continue to use them as examples.

![img_kids_family_google_play_old](Images/Google_Play_Kids_Family_old.png)

The updated "Kids" category
![img_kids_family_google_play_new](Images/Google_Play_Kids_Family_new_2025.png)

Even so, practical apps seem to have a better representation on Google Play compared to the iOS App Store. These observations are also confirmed by the frequency table for the `Genres` column:

In [45]:
android_genres = freq_table(android_english_free, 9)
# print(android_genres[:15])

display_sorted_table(android_genres)

Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.91
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;Action & Adventure : 0.17
Puzzle;Brain Games : 0.17
Entertainment;Music & Video : 0.17
Casual;

![img_Genres_Google_Play](Images/Google_Play_Genres.png)

The differences between the `Category` and the `Genres` columns are yet to be investigated. As mentioned beforehand, the `Category` column is much less granular than the `Genres` column (`Genres` is much more numerous; note that about half of them are grouped under "Remaining" in the graph to save some space). This confirms our previous intention that we are good to proceed with just the `Category` column, given we're currently looking only at the bigger picture.

Up to this point, we can summarize that when it comes to free English apps, the iOS App Store is heavily dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'll focus on determining the kind of apps that have the most users, which is the main goal of this project.

## Most popular apps by Genre

### Basic overview

Let's quickly explore the columns from both datasets, to see which can be useful in determining the popularity of the apps. We can define popularity as the number of people that used the apps, and we can combine this with the number of people that liked the apps to give us better clarity. 

| Google Play | iOS App Store    |
|-------------|------------------|
| App (name)  | track_name       |
| Category    | prime_genre      |
| Genres      | prime_genre      |
| Reviews     | rating_count_tot |
| Reviews     | rating_count_ver |
| Rating      | user_rating      |
| Rating      | user_rating_ver  |
| Installs    | n/a              |
| Price       | price            |
| Type (Free or Paid) | n/a (can use price amount) |
| n/a         | currency         |

For Google Play, we can use up to three of the columns: the `Installs` column showing how many users downloaded/installed the app; the `Reviews` column which shows how many users reviewed the app; and the `Rating` column showing the overall (average) user rating for that app, in case we want to take into consideration the positive vs negative reviews.

| Column   Title | Explanation |
|---|:---|
| App | Application name |
| Installs | Number of user downloads/installs for the app |
| Reviews | Number of user reviews for the app |
| Rating | Overall user rating of the app |

We could focus on the `Installs` column and find the average for each app genre, but this does not take into consideration how many of the users actually enjoyed the app, nor how many uninstalled it shortly after or kept it installed but not really opened it much. The column is also missing for the iOS dataset. As a workaround, we can take the total number of user reviews as a proxy for both datasets, which will also balance out the analysis. For Google Play, this will be the `Reviews` column. We can then additionally analyse the `Rating` column to get more information.

Below are 3 examples in a table (we took rows indexed 16-18 because they had shorter app names). Just as a reminder, all numbers and dates represented in the tables below are from the time of scrapping and might not necessarily reflect the current state.

**Google Play sample**

| App | Category | Rating | Reviews | Size | Installs | Type | Price | Content Rating | Genres | Last Updated | Current Ver | Android Ver |
|:--|:--|---|---|---|---|:--|---|:--|:--|---|---|---|
| 350 Diy   Room Decor Ideas | ART_AND_DESIGN | 4.5 | 27 | 17M | 10,000+ | Free | 0 | Everyone | Art & Design | November 7, 2017 | 1 | 2.3 and up |
| FlipaClip - Cartoon animation | ART_AND_DESIGN | 4.3 | 194216 | 39M | 5,000,000+ | Free | 0 | Everyone | Art & Design | August 3, 2018 | 02.2.2005 | 4.0.3 and up |
| ibis Paint X | ART_AND_DESIGN | 4.6 | 224399 | 31M | 10,000,000+ | Free | 0 | Everyone | Art & Design | July 30, 2018 | 05.5.2004 | 4.1 and up |

For the iOS dataset, we can choose between two columns showing the number of user reviews, `rating_count_tot` referring to all versions of the app, and the alternative `rating_count_ver` column, which is limited because it counts the user reviews only for the current version, and its values are much smaller than the total column as demonstrated in the examples below. Using the same logic, we can take the `user_rating` column if we want to consider the positive vs negative reviews; this columns shows the average user rating value for all versions of the app.

| Column   title | Explanation |
|---|:---|
| id | App ID |
| track_name | App Name |
| rating_count_tot | User Rating counts (for all version) |
| rating_count_ver | User Rating counts (for current version) |
| user_rating | Average User Rating value (for all version) |
| user_rating_ver | Average User Rating value (for current version) |

**iOS sample**

| id | track_name | size_bytes | currency | price | rating_count_tot | rating_count_ver | user_rating | user_rating_ver | ver | cont_rating | prime_genre | sup_devices.num | ipadSc_urls.num | lang.num | vpp_lic |
|:--|:--|---|---|---|---|---|---|---|---|---|:--|---|---|---|---|
| 284882215 | Facebook | 389879808 | USD | 0 | 2974676 | 212 | 3.5 | 3.5 | 95 | 4+ | Social Networking | 37 | 1 | 29 | 1 |
| 389801252 | Instagram | 113954816 | USD | 0 | 2161558 | 1289 | 4.5 | 4 | 10.23 | 12+ | Photo & Video | 37 | 0 | 29 | 1 |
| 529479190 | Clash of Clans | 116476928 | USD | 0 | 2130805 | 579 | 4.5 | 4.5 | 9.24.12 | 9+ | Games | 38 | 5 | 18 | 1 |

To minimize bias and balance out the analysis for both datasets, given our limited choices, the main focus will be on the number of user reviews for both datasets as proxies for their popularity: `Reviews` for the Google Play dataset (assuming this counts the user reviews for all versions of the app), and `rating_count_tot` for the iOS dataset. It wouldn't be reasonable to compare `Installs` for Google Play vs `rating_count_tot` for the iOS App Store. Just by looking at the Google Play examples, we can see the number of installs is much higher than the number of reviews, making the beforementioned comparison highly skewed.

Though we cannot be certain that every review comes from a unique user, common sense and experience suggest that it's not very likely a user would leave a review more than once (we've encountered real-life examples where users edited their existing reviews after using the app some more). They might also leave multiple reviews for different versions of the app. But either way, there probably wouldn't be more than a few reviews coming from the same user, for example 2-3 reviews per user, given people don't tend to leave reviews all the time. As we've seen from the sample table above, not all people that install the apps leave reviews for them. After all, it requires putting in some effort without getting anything tangible in return, when uninstalling the app is quicker and will do just the job. Even if users leave multiple reviews, both of the datasets might balance each other out anyway in terms of how many unique users reviewed the apps, given the size of each dataset.

Having said that, the number of user reviews doesn't necessarily represent the popularity of the apps either, since they can be both positive and negative, although even a negative review means someone used the app at some point (though probably not for long). In addition to that, an app with 10,000 negative reviews is still more popular than an app with 100 positive reviews, and on the other hand negative reviews could intrigue others to see if the app is truly that bad and drive some installs, but that's debatable at this point. To be on the safe side, we can consider a combination of the number of reviews and the proportion of positive reviews.

Alternatively, if there were more columns to choose from, we might've been able to do a more reliable analysis with the number of active users, combining the number of people that installed/downloaded the app with the duration of usage, for example: time between install and uninstall (if it occured), the number of opens, the time they spent interacting with the app (time the app was opened on the screen, the number of clicks), and so on. And of course, it would be very useful if we could have information about how they respond to ads and which types of ads were shown, but we'll need to gather more data for that.

Anyway, let's get back to analyzing the number or reviews columns. And as mentioned before, we can consider also the proportion of positive vs negative reviews to get a clearer picture. Last but not least, we also must utilize the genre columns for both datasets, since we're interested in the most popular apps by genre. This leaves us with the following columns:

|| Number of user reviews/ratings | Average value of user ratings | Genre |
| --------------- | :--------------- | :---------- | :------------------- |
| **Google Play** | Reviews          | Rating      | Category (or Genres) |
| **iOS**         | rating_count_tot | user_rating | prime_genre          |

### Popular iOS apps using the number of reviews

In this section, we'll find the average number of user reviews per genre. We'll use the existing frequency table `ios_prime_genres`, given it already contains the unique genres of the dataset.

In [46]:
print(ios_prime_genres)

{'Social Networking': 3.29, 'Photo & Video': 4.97, 'Games': 58.14, 'Music': 2.05, 'Reference': 0.56, 'Health & Fitness': 2.02, 'Weather': 0.87, 'Utilities': 2.52, 'Travel': 1.24, 'Shopping': 2.61, 'News': 1.34, 'Navigation': 0.19, 'Lifestyle': 1.58, 'Entertainment': 7.89, 'Food & Drink': 0.81, 'Sports': 2.14, 'Book': 0.43, 'Finance': 1.12, 'Education': 3.66, 'Productivity': 1.74, 'Business': 0.53, 'Catalogs': 0.12, 'Medical': 0.19}


To find the average, we need to do the following: isolate the apps for each genre, sum the number of user reviews for the apps of that genre, and divide that sum by the number of apps belonging to that genre (not by the total number of apps in the dataset). We'll use nested loops (`for` loop inside of another `for` loop) and build a function right away.

The function will include options to choose between the sum or the average number of reviews per genre, and an optional threshold parameter, which whill allow us to filter out apps where the number of reviews is higher than a certain threshold, in other words apps that are relative outliers within the genre and skew the averages. This parameter will rely on `continue` to skip these apps. We originally built the function with specific thresholds in mind for the iOS dataset, filter200K and filter100K, for which we'll explain the reasoning behind it a bit later. But to avoid repetion of code and to make the function reusable for both datasets, we introduced the threshold parameter instead. This will allow us to use the appropriate numbers as thresholds; for Google Play, 200K and 100K might not be suitable to be considered outliers for the number of reviews per app, given that the cleaned up Google Play dataset with 8,864 total apps is likely to have more reviews overall than the iOS dataset with 3,220 apps. The function will also exclude apps with 0 reviews, to get more realistic averages.

Note that the first two parameters of the function come in pair, where we need to choose *one of* the options, ie. we cannot choose both the sums and the averages. The same applies for the alternative filter200K and filter100K parameters (using the 100K+ filter will exclude apps with 200K+ reviews anyway). When it comes to the sums vs averages, it's better to use averages, because with sums we lose information for how many apps there are within a genre, and the number of apps will not be proportional between the genres. Meanwhile, the averages consider the number of apps in each genre and provides a good basis for comparison.

In [47]:
# The dataset argument should be either ios_prime_genres or android_category

def avg_number_reviews(dataset, sum_reviews=False, avg_reviews=False, 
                       threshold=None): # filter200K=False, filter100K=False
    
    avg_number_reviews_genre = {}
    dataset2 = None
    genre_dataset = None
    number_reviews_dataset = None
    
    if dataset == ios_prime_genres:
        dataset2 = ios_english_free
        genre_dataset = 11 # prime_genre
        number_reviews_dataset = 5 # rating_count_tot
        
    elif dataset == android_category:
        dataset2 = android_english_free
        genre_dataset = 1 # Category
        number_reviews_dataset = 3 # Reviews
    
    for genre in dataset:
        sum_number_reviews = 0
        sum_apps = 0 # number of apps

        for app in dataset2:
            genre_app = app[genre_dataset]
            number_reviews = int(app[number_reviews_dataset])
            
            if genre_app != genre:
                continue
            if number_reviews == 0:
                continue # excluding apps with 0 reviews

            if threshold is not None and number_reviews >= threshold:
                continue # skip this app, it's too popular
            
            ### code for using filter200K=False, filter100K=False parameters:
            """
            if filter200K and not filter100K and number_reviews >= 200000:
                continue # skip this app, it's too popular
            elif filter100K and number_reviews >= 100000:
                continue # skip this app, also too popular
            """
                
            sum_number_reviews += number_reviews
            sum_apps += 1

        if avg_reviews and not sum_reviews:
            avg_num_reviews = round(sum_number_reviews / sum_apps)
            avg_number_reviews_genre[genre] = avg_num_reviews
        
        elif sum_reviews and not avg_reviews:
            avg_number_reviews_genre[genre] = sum_number_reviews
    
    return display_sorted_table(avg_number_reviews_genre)

avg_number_reviews(dataset=ios_prime_genres, avg_reviews=True)

Navigation : 86090
Reference : 84310
Social Networking : 75090
Weather : 58553
Music : 58208
Book : 55662
Food & Drink : 33334
Finance : 33319
Travel : 30534
Photo & Video : 30137
Shopping : 27917
Health & Fitness : 25667
Sports : 24055
Games : 23778
News : 23427
Productivity : 22219
Utilities : 20452
Lifestyle : 17159
Entertainment : 14545
Business : 7491
Education : 7446
Catalogs : 4004
Medical : 918


In terms of free English apps, when nothing is filtered yet, the iOS App Store shows the highest average number of user reviews per app for the Navigation genre, followed by Reference (used to access or retrieve information) and Social Networking. These categories are somewhat similar, in the sense they're used for either finding something or communicating with someone. Along these lines, Weather also has quite a lot of reviews. 

Since these are averages, there's a possibility only a handful of apps within a genre drive up the averages by a lot. Let's check if that's the case here.

In [48]:
for app in ios_english_free:
    if app[11] == "Navigation": # switch to explore different genres
        print(app[1], ':', app[5]) # name and number of user reviews

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The primary category, Navigation, is heavily influenced by Waze and Google Maps, which have close to half a million user reviews combined. Likewise, the Reference category is skewed by the Bible and Dictionary.com apps, where the Bible itself has almost a million reviews. The Social Networking genre is influenced mostly by Facebook and Pinterest, which together have about 4 million user reviews, but there are also other widely-known apps that have plenty of reviews, such as Skype, Messenger, Tumblr, WhatsApp, and so on. We could say that this might be the case for each genre, including those with lower average number of user reviews.

The aim is to find popular genres, but Navigation, Reference, or Social Networking might seem more popular than they really are. The average number of user reviews seems to be skewed by very few apps with hundreds of thousands of reviews, some even reaching millions, while the rest of the apps may struggle to get past the 100K threshold. We could get a better picture by filtering out these extremely popular apps for each genre and recalculate the averages, but we'll come back to this later.

Even with all of these predominant apps, it doesn't have to mean that the company should avoid these genres. If it wants to aim high, it can try creating an app in Social Networking or Music genres and offer something that the competitors are missing; there are still other apps that have a solid number of user reviews, and who knows, maybe the company will develop the next big giant. Besides that, there are other things to consider, such as which genres are more suitable for ads, and which genres don't have high barriers to entry (which wouldn't be as much of a problem if the company already has experience or domain knowledge). 

Nevertheless, let's explore some of the high-performing categories:

* Navigation and Weather apps - people generally don't spend too much time in-app, therefore the chances of making profit from in-app adds might be low. Also, getting reliable navigation or live weather data may require incurring additional expenses and seeking out partnerships.


* Reference - according to the [Apple website](https://developer.apple.com/app-store/categories/), this category is about "Apps that assist the user in accessing or retrieving information. For example: atlas, dictionary, thesaurus, quotations, encyclopedia, general research, animals, law, religion, how-tos, politics." In other words, helping users finding answers to some questions. Having ads here can be tricky; if they are not well targeted, they might annoy users or even get completely ignored. Presenting new, unwelcomed information while trying to retrieve other specific information might contribute to infoglut and overwhelm the users. The same could be said about Navigation and Weather apps, but the slight advantage there is that users already know what they can expect to see because the type of information is consistent (it's always a map, just a different direction), which reduces the infoglut.


* Social Networking - this genre is heavily dominated by very well-known apps, and the opportunities to gather a sustainable large base of users might be limited. The best way to stand out might be to offer certain functionalities that are more innovative or the competitors are simply missing, or try to reach a more niche audience that is large enough to make it profitable. But knowing how difficult it can become to compete with the big fish, especially in this genre with apps such as Facebook, Pinterest and Instagram (placed in the Photo & Video genre), that are probably among the most known companies in the world and whose apps have more than 1 million reviews each, might not be worth the investment and time, especially since it's not uncommon for them to buy out smaller companies or even apply unfair practices.


* Music - these apps would require a library of existing audio or video formats created by various artists. This probably requires dealing with servers, copyright, etc. It also doesn't seem to be worth it, as there are already widely-used apps; offering a different app to listen to the same music seems futile, while offering a music app with niche music genres will probably not attract many users. In addition to that, music lovers can vouch how annoying ads can be while playing music.


* Books - There isn't as much competition in this genre, there are only 14 apps with Kindle and Audible having over 200K and 100K reviews each. But this genre doesn't offer many options to differentiate either, and reading books might not be the best place to place an ad because it might distract and annoy the reader.


* Food & Drink - examples here include Starbucks, Domino's, Doordash,  UberEATS, McDonald's, etc. This means the publishers of these apps are either the owners of, or partnered with, actual cooking and delivery services, which is outside the scope of the company. There also seem to be some general delivery apps, but similar to the Music genre, it might be futile to develop them. What could work is a recipe or a food knowledge app that advertises related products or servives. The same can be said for the Shopping genre, which seems relatively saturated with apps.


* Finance - these apps involve not only dealing with very sensitive user information, but also domain expertise knowledge. Building such an app might require hiring a finance expert, which might not be worth the money. These apps include banking, investment, bill management, small business finance, etc. Alternatively, the Business and Productivity genres might be more suitable and more generic and provide more opportunities for effective ads.


* Travel - This seems like a good genre for ads. While users are planning their travel, they might benefit from relevant ads or practical items they could use while abroad. It can also be combined with blogs, connecting with fellow travellers, reviews of services, video and audio guides, and even with the Photo & Video and Education genres to enrich the experience and international knowledge.


* Health & Fitness - this seems like another great place for relevant ads, alongside the Lifestyle genre, eg. dietary suggestions and physical activities, as well as real estate, hobbies, fashion etc. for the latter. Both genres are somewhat saturated, but they offer a bigger variety and relatively lower competition with giants: there are only 3 apps with about 300-500K reviews per app, everything else is about 100K reviews or under.

From the examples above, we can see that the number of user reviews, or the number of users in general, is not the only thing to consider in building a free app with ads. Any app where users don't spent too much time per usage or might get infoglutted by ads; apps that require additional expenses, servers, partnerships, licences, specific domain knowledge etc; apps with very niche target audiences; or more generic apps that already have well-established competitors or where the market is overly saturared - might not be the best option for developing a new, free app with ads.

Generally, it seems like a good idea to develop a free app in genres where users to go search for things that can technically be bought, and promote related products and services, such as Reference, Food & Drink, Business, Productivity, Travel, Health & Fitness and Lifestyle. It might be easier to find relevant advertisers with ads that won't overwhelm or annoy users. It also doesn't seem they require too specific domain knowledge unlike other genres. These genres can also allow for variety between the apps, easing the oversaturation problem, allowing for creativity and differentiation, and even combining multiple genres.

This idea seems to fit well with the fact that the iOS App Store is generally saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the large number of apps on the iOS App Store, and it would also fit well with the Google Play market.

It is probably a safe bet to create a Game or Entertainment app in terms of popularity and profitability, but these genres have the highest number of apps on the iOS App Store, with two thirds of all apps belonging to these genres. Besides, when it comes to Games, it would require human resources specifically skilled in game development.

### Filtering out iOS apps with more than 100K and 200K reviews

Let's see how the averages change when we filter out the extremely popular apps, aka apps with over 200K user reviews. First we'll use the code below to get the number of reviews per app and include the genre. We can further explore each genre by using Excel or Google Sheets.

In [148]:
ios_sample_genre_reviews = []

for app in ios_english_free:
    genre_app = app[11] # prime_genre
    name = app[1]
    number_reviews = int(app[5]) # rating_count_tot
    
    # print(genre_app, ";", name, ";", number_reviews)
    ios_sample_genre_reviews.append([genre_app, name, number_reviews])

for row in ios_sample_genre_reviews[:5]:
    print(row)

['Social Networking', 'Facebook', 2974676]
['Photo & Video', 'Instagram', 2161558]
['Games', 'Clash of Clans', 2130805]
['Games', 'Temple Run', 1724546]
['Music', 'Pandora - Music & Radio', 1126879]


For the iOS dataset, there are only a few apps that have more than a million user reviews. We will definitely remove those, but it might not be enough; by exploring the genres, we see some apps with over 100K reviews that stand out from the rest within the genre, because that genre as a whole has relatively lower number of reviews compared to other genres. In that sense, not all genres have the same outliers; for some an outlier will be an app with over 1m reviews, for others it will be over 10K reviews. This leaves us with four options: we can keep the outliers, remove the relative outliers per genre, remove the outliers based on percentages, or simply remove apps where the reviews go over a number threshold. While some of these methods might not be statistically sound, they can help us filter out the apps that drive the averages by a lot. 

Removing them by percentages seems like a good idea, for examle apps that hold more than 20% of the total reviews for the genre, but this includes removing some apps with less than a 100K reviews - 1K, 13K and 38K reviews - which are not high numbers compared to apps from other genres. These genres don't have many reviews anyway, so a 38K number is relatively high compared to other apps within the same genre. However, the rest of the apps with percentage over 20% almost all have more than 200K user reviews (including one app with 154K), which we can simply take as the threshold for all apps throughout all the genres. If we go by this standard, we'd remove 81 apps total, which is not too many. We could also opt for the 100K threshold, as the relative outliers for many of the genres are beyond 100K reviews. But this would mean removing 2,048 apps in total, which is too many, so maybe it's better to use the 100K filter just for the genres where it's a relative outlier.

Anyway, let's filter out apps with more than 200K reviews, then try removing those with over 100K reviews.

In [57]:
avg_number_reviews(dataset = ios_prime_genres, 
                   avg_reviews = True, 
                   threshold = 200000)

Navigation : 34299
Book : 33838
Weather : 33024
Productivity : 22219
Finance : 20832
Social Networking : 20638
Shopping : 20016
Sports : 19948
Music : 17737
News : 14726
Utilities : 14164
Travel : 13134
Food & Drink : 12675
Games : 12278
Photo & Video : 12074
Reference : 11642
Entertainment : 11220
Health & Fitness : 11102
Lifestyle : 10371
Business : 7491
Education : 7446
Catalogs : 4004
Medical : 918


These averages are significantly lower than the unfiltered averages for all apps. Navigation still has the highest average number of reviews, but the second place is taken by the Books genre, followed by Weather, Productivity and Finance, out of which Productivity moves up significantly in the ranking. Reference moves down by quite a lot, Music, Travel, Food & Drink and Health & Fitness also move down, while Social Networking is still ranked quite high. We can conlude that there is definitely a change in the order of the ranking, with some apps going up and some going down, though most of them change by a few places. Most notable changes are Books taking the second place, Productivity increasing by 12 places, and Reference decreasing by 14, which is a significant change.

When we filter out apps with over 100K user reviews, the order changes yet again. But remember that 100K is not necessarily an outlier for some of the genres with higher number of user reviews overall, eg. the Games genre.

In [58]:
avg_number_reviews(dataset = ios_prime_genres, 
                   avg_reviews = True, 
                   threshold = 100000)

Book : 24909
Social Networking : 14672
Productivity : 13136
Shopping : 13113
Reference : 11642
Finance : 11231
Sports : 10821
Photo & Video : 9896
Utilities : 9544
Travel : 9239
News : 8723
Weather : 8632
Entertainment : 8574
Music : 8545
Business : 7491
Games : 7171
Health & Fitness : 7117
Lifestyle : 5528
Education : 4959
Navigation : 4146
Catalogs : 4004
Food & Drink : 3678
Medical : 918


Books now take the first place. Navigation which used to hold the 1st place, is now hear the very bottom with only 4,146 average user reviews per app; this is because Google Maps (154K) is removed in addition to the Waze app (345K), leaving only 4 other apps with under 13K reviews each. Food & Drink comes near the bottom as well, and Weather decreases significantly. Social Networking takes the second place, Productivity remains near the top, and Reference comes back up and takes the 5th place. Shopping moves up gradually. Finance, Travel and Health & Fitness remain relatively stable compared to the 200K filter, and Business moves up slightly.

This goes to show that the method and the treshold we use to filter out outliers definitely impacts the averages, and indicates that we need to be extra careful in choosing what to use.

Now let's analyze the Google Play market.

### Popular Google Play apps using the number of reviews

To keep things consistent, let's focus on the `Reviews` column for the Google Play dataset, and reuse the `avg_number_reviews` function that we created in the iOS section. Besides changing the dataset argument, we'd also need to look into the thresholds for filtering the apps by number of reviews and adjust them if needed, given that the cleaned up Google Play dataset with 8,864 total apps is likely to have more reviews than the iOS dataset with 3,220 apps. But right now, let's look at the unfiltered dataset.

Just a reminder that we need to use either the avg or the sum parameters, we cannot use both at the same time.

In [59]:
avg_number_reviews(dataset = android_category, avg_reviews = True)

COMMUNICATION : 1094788
SOCIAL : 1013049
GAME : 690736
VIDEO_PLAYERS : 436327
PHOTOGRAPHY : 410371
TOOLS : 320699
ENTERTAINMENT : 301752
SHOPPING : 230848
PERSONALIZATION : 195055
WEATHER : 178806
PRODUCTIVITY : 178196
MAPS_AND_NAVIGATION : 146402
TRAVEL_AND_LOCAL : 137453
SPORTS : 126160
FAMILY : 117054
NEWS_AND_MAGAZINES : 99508
BOOKS_AND_REFERENCE : 94458
HEALTH_AND_FITNESS : 84940
FOOD_AND_DRINK : 62601
EDUCATION : 56293
COMICS : 42586
FINANCE : 39873
LIFESTYLE : 37379
BUSINESS : 29896
HOUSE_AND_HOME : 28379
ART_AND_DESIGN : 25140
DATING : 24475
PARENTING : 16379
AUTO_AND_VEHICLES : 14865
LIBRARIES_AND_DEMO : 11196
BEAUTY : 7769
MEDICAL : 4508
EVENTS : 2776


The Communication and Social categories are at the top with over 1M number of reviews each, followed by Game, Video Players and Photography. The next few with the exception of Entertainment are more practical-oriented apps. Some of them seem to be similar to the genres from the iOS dataset, and some have similar ranking as well, such as Communication and Social with Social Networking from iOS. Let's explore what the results look like by filtering out outliers and using Google Sheets for assistance.

In [121]:
android_sample_category_reviews = []

for app in android_english_free:
    genre_app = app[1] # Category
    name = app[0]
    number_reviews = int(app[3]) # Reviews
    
    # print(genre_app, ";", name, ";", number_reviews)
    android_sample_category_reviews.append([genre_app, name, number_reviews])

for row in android_sample_category_reviews[:5]:
    print(row)

['ART_AND_DESIGN', 'Photo Editor & Candy Camera & Grid & ScrapBook', 159]
['ART_AND_DESIGN', 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 87510]
['ART_AND_DESIGN', 'Sketch - Draw & Paint', 215644]
['ART_AND_DESIGN', 'Pixel Draw - Number Art Coloring Book', 967]
['ART_AND_DESIGN', 'Paper flowers instructions', 167]


By using the percentage method, we identified only 20 apps throughout the entire database with reviews that compromise more than 20% of the total within the category. The maximum is 66-78M reviews for 3 apps in the top 2 categories: Facebook, WhatsApp and Instagram. These are followed by YouTube with 25M reviews in the Video Players category, and a few more with a couple of million reviews: Twitter with 11M (News and Magazines), Maps with 9M (Travel and Local), Waze with 7M (Maps and Navigation), Netflix with 5M (Entertainment), Uber with 4.9M (Maps and Navigation), etc. Most of these are widely internationally known apps, and some repeat as outliers in the iOS dataset as well. The rest of the 20%+ apps have under 700K reviews each, and only 2 apps have under 200K reviews.

Generally, some categories can use the 100K or 200K threshold, for others it's more suitable to use over 1M reviews like Books and Personalization, or even 10M like Communication, Social and Game. To keep things simple, we'll use the 1M and 10M thresholds and see how the rankings of the category averages change.

### Filtering out Google Play apps with more than 1M and 10M reviews



In [67]:
avg_number_reviews(dataset = android_category, 
                   avg_reviews = True, 
                   threshold = 1000000) # 1M

ENTERTAINMENT : 121392
PHOTOGRAPHY : 120978
GAME : 117292
VIDEO_PLAYERS : 91660
SHOPPING : 78358
SOCIAL : 75005
COMMUNICATION : 71591
PRODUCTIVITY : 63656
SPORTS : 63155
HEALTH_AND_FITNESS : 59787
BOOKS_AND_REFERENCE : 57069
WEATHER : 54612
TRAVEL_AND_LOCAL : 53189
FOOD_AND_DRINK : 52897
TOOLS : 49685
EDUCATION : 45983
PERSONALIZATION : 45838
FAMILY : 45503
NEWS_AND_MAGAZINES : 44063
MAPS_AND_NAVIGATION : 35015
FINANCE : 31521
HOUSE_AND_HOME : 28379
ART_AND_DESIGN : 25140
COMICS : 24598
DATING : 24475
LIFESTYLE : 21529
BUSINESS : 19352
PARENTING : 16379
AUTO_AND_VEHICLES : 14865
LIBRARIES_AND_DEMO : 11196
BEAUTY : 7769
MEDICAL : 4508
EVENTS : 2776


By filtering out the apps with 1M+ reviews, Entertainment and Photography take the first and second spot respectively. Communication and Social both move a bit downwards. Game and Video Players remain at the exact same place, and a few more remain stable. Tools, Personalization and Maps and Navigation move down, while Productivity, Books and Reference, Health and Fitness and Sports move up. Now let's check with the 10M threshold.

In [68]:
avg_number_reviews(dataset = android_category, 
                   avg_reviews = True, 
                   threshold = 10000000) # 10M

GAME : 474459
COMMUNICATION : 381124
PHOTOGRAPHY : 369555
ENTERTAINMENT : 301752
SOCIAL : 298131
VIDEO_PLAYERS : 272567
SHOPPING : 230848
PERSONALIZATION : 195055
WEATHER : 178806
PRODUCTIVITY : 178196
TOOLS : 166860
MAPS_AND_NAVIGATION : 146402
TRAVEL_AND_LOCAL : 137453
SPORTS : 126160
FAMILY : 104671
BOOKS_AND_REFERENCE : 94458
HEALTH_AND_FITNESS : 84940
FOOD_AND_DRINK : 62601
EDUCATION : 56293
NEWS_AND_MAGAZINES : 49430
COMICS : 42586
FINANCE : 39873
LIFESTYLE : 37379
BUSINESS : 29896
HOUSE_AND_HOME : 28379
ART_AND_DESIGN : 25140
DATING : 24475
PARENTING : 16379
AUTO_AND_VEHICLES : 14865
LIBRARIES_AND_DEMO : 11196
BEAUTY : 7769
MEDICAL : 4508
EVENTS : 2776


Game and Communication now come up to the top. The first 10 categories remain pretty much the same as the unfiltered ranking and the 1M ranking. We can use these as guidance for the recommendations. For most of these, we already elaborated in the bullet points in the iOS section, which can apply here as well. We'll only expand on a few more:

* Sports - This is a niche category, which does have a lot of potential given the number of sports fans around the world. However, the ads should be really specific to the app and even the sport itself to make them relevant, eg. anatomic shoes, equipment, or energy drinks. It would make more sense for the company to work on this kind of app only if it has the needed knowledge and interest in Sports, to match the knowledge and enthusiasm of the fans. The same can actually be said for the Comics category, but the average number of user reviews is not that high.


* Personalization - These are mainly customization apps such as wallpapers, themes, APK launchers, ringtones, etc. Given users probably don't spent significant amount of time on them, it's not worth the investment for developing an app with ads. It's probably the similar case with Tools, which include VPN, translation apps, flashlights and other practical apps that would likely run in the background.


* Shopping - This was mentioned with the Food & Drinks genre in the iOS section. Similar to that genre, the publishers of these apps are probably e-commerce companies that operate their own websites, and developed an app for their mobile users. The most dominant apps in this category are some established brands like Wish, AliExpress, eBay, Flipkart and Mercado Libre. Developing an app wouldn't really make sense here because the company is not an e-commerce company, and the apps by design are driven by revenue that comes directly from the shoppers. What might work here is ideas for shopping with links to the specific products, or an aggregator for reviews suggesting better alternatives where applicable.


* News and Magazines - This seems suitable for ads, given that users are already open to receiving new information when they open the app, and there's a variety of topics that can be covered, and hence a variety of ads. But it might be a tough place to compete with established media companies like BBC or popular apps like Reddit and Twitter dominating the market in this area. Niche audiences might be more profitable here, but might not be sizeable enough to make it sustainable in the long run.

The Shopping, News and Magazines, and Sports categories follow the same general idea explained in the iOS section, ie. developing free apps in genres where users to go search for things that can technically be bought, and find relevant advertisers that promote directly related products and services which will minimize the risk of pushing the users away. However, Sports is niche, Shopping and News and Magazines are difficult to compete with, and there might be some limitations with respect to creativity and differentiation, unlike other categories in the top.

### Popular Google Play apps using the number of installs

Given we have data about the number of installs for the Google Play market, and that we want to present solving different challenges for our demonstrations purposes than the `Reviews` column and the `rating_count_tot` for iOS, we will additionally try out the `Installs` column. This will broaden the picture for the popularity of the categories (or genres). The comparison with iOS number of installs (that's currently missing) is not entirely necessary at this point anyway, since the company wants to develop an app for the Google Play market first.

The install numbers don't seem precise enough and are not formatted like numbers - we can see that most values are open-ended and have a plus sign (100+, 1,000+, 5,000+, etc.):

In [93]:
android_installs_number = freq_table(android_english_free, 5, percentages=False)

print(android_installs_number)

{'10,000+': 904, '5,000,000+': 605, '50,000,000+': 204, '100,000+': 1024, '50,000+': 423, '1,000,000+': 1394, '10,000,000+': 935, '5,000+': 400, '500,000+': 493, '1,000,000,000+': 20, '100,000,000+': 189, '1,000+': 744, '500,000,000+': 24, '500+': 288, '100+': 613, '50+': 170, '10+': 314, '1+': 45, '5+': 70, '0+': 4, '0': 1}


The issue with the data not being precise means that, for instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to find out which app genres attract the most users, and we don't need perfect precision with respect to the number of installs.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, an app with 1,000,000+ installs has 1,000,000 installs, and so on. To perform computations, however, we'll need to convert each install number from a string to a float. This means we need to remove the thousand commas and the plus characters, otherwise the conversion will fail and raise an error.

To remove characters from strings, we can use the `str.replace(old, new)` method. We'll do this directly in the loop further down, where we'll also calculate the average number of installs for each `Category`. We'll use the existing `android_category` frequency table, given it already provides the unique app `Category` for the Google Play dataset. Finally, we'll need to use a nested loop, similar to the other dataset.

Following the example with the iOS dataset, let's explore if any of the apps have 0 installs. We'll remove these later to get more realistic averages.

In [94]:
display_sorted_table(android_installs_number)

# Alternative code without the values

# unique_installs = []

# for app in android_english_free:
#     installs = app[5]
#     if installs not in unique_installs:
#         unique_installs.append(installs)
#         print(installs)

1,000,000+ : 1394
100,000+ : 1024
10,000,000+ : 935
10,000+ : 904
1,000+ : 744
100+ : 613
5,000,000+ : 605
500,000+ : 493
50,000+ : 423
5,000+ : 400
10+ : 314
500+ : 288
50,000,000+ : 204
100,000,000+ : 189
50+ : 170
5+ : 70
1+ : 45
500,000,000+ : 24
1,000,000,000+ : 20
0+ : 4
0 : 1


This one is a bit tricky. There are 2 values referencing 0, but one of them has a plus sign, while the other one doesn't. For the one that doesn't, it's safe to say those apps have 0 installs. But for the one with the plus, we might think at first that those have more than 0 installs, which could be correct, but then there's the next value of 1+ which could either mean more than 1 installs (aka 2 or above), or 1 and above. If it's the latter, 0+ shouldn't technically be different than 0, but then it wouldn't make sense to have those values as different. So in this case, we will assume that the apps with 0+ installs do have at least one install, and keep them. Either way, the frequency table above shows that there are only 5 apps with 0 or 0+ installs in the cleaned up Google Play dataset. To be on the safe side, we can investigate if they have any reviews and ratings, and check the price (type: free or paid) and the category to get more context.

In [122]:
print(android_header[0], android_header[1], android_header[2],
      android_header[3], android_header[5], android_header[6],
      android_header[9], "\n")

for app in android_english_free:
    installs_app = app[5]

    if installs_app == "0+" or installs_app == "0":
        print(app[0], app[1], app[2], app[3], app[5], app[6], app[9])

App Category Rating Reviews Installs Type Genres 

Pekalongan CJ SOCIAL NaN 0 0+ Free Social
CX Network BUSINESS NaN 0 0+ Free Business
Sweden Newspapers NEWS_AND_MAGAZINES NaN 0 0+ Free News & Magazines
Test Application DT 02 ART_AND_DESIGN NaN 0 0+ Free Art & Design
Command & Conquer: Rivals FAMILY NaN 0 0 NaN Strategy


| App | Category | Rating | Reviews | Installs | Type | Genres |
| :-- | :-- | --- | --- | --- | --- | :-- |
| Pekalongan CJ | SOCIAL | NaN | 0 | 0+ | Free | Social |
| CX Network | BUSINESS | NaN | 0 | 0+ | Free | Business |
| Sweden Newspapers | NEWS_AND_MAGAZINES | NaN | 0 | 0+ | Free | News & Magazines |
| Test Application DT 02 | ART_AND_DESIGN | NaN | 0 | 0+ | Free | Art & Design |
| Command & Conquer: Rivals | FAMILY | NaN | 0 | 0 | Free | Strategy |

As suspected, the apps have 0 reviews and no ratings, which doesn't necessarily mean there were no installs at all (they might've installed the app but leave no review/rating). Seeing that there are plenty of apps with more installs but no reviews or ratings in the code below, supports this statement. The apps are all free, which means the price wasn't an obstacle to installing. Even with that, they might have a higher number now that several years have passed since the dataset was provided on Kaggle. One of them even has "Test" in the name, and others might've been relatively new at the time of the acquiring the data. Having said all of this, we will exclude the apps with 0 or 0+ installs, given they're very few and don't have any reviews. We'll also update the code above and include the number of the apps with no reviews.

In [142]:
android_zero_reviews = []

# print(android_header[0], android_header[1], android_header[2],
#       android_header[3], android_header[5], android_header[6],
#       android_header[9], "\n")

for app in android_english_free:
    name = app[0]
    category = app[1]
    rating = app[2]
    reviews = app[3]
    installs = app[5]
    free_paid = app[6]
    genres = app[9]

    if reviews == "0": # or rating = "NaN"
        android_zero_reviews.append([name, category, rating, reviews, 
                                     installs, free_paid, genres])

print(len(android_zero_reviews), "apps with no reviews")
for row in android_zero_reviews[:5]:
    print(row)

516 apps with no reviews
['Dating White Girls', 'DATING', 'NaN', '0', '50+', 'Free', 'Dating']
['Geeks Dating', 'DATING', 'NaN', '0', '50+', 'Free', 'Dating']
['CAM5678 Video Chat', 'DATING', 'NaN', '0', '500+', 'Free', 'Dating']
['Video chat live advices', 'DATING', 'NaN', '0', '100+', 'Free', 'Dating']
['Pet Lovers Dating', 'DATING', 'NaN', '0', '10+', 'Free', 'Dating']


Now that that's settled, we'll start building the function, and exclude the apps with 0 installs within. We'll also remove the plus signs and the thousand commas, which will allow us to do calculations and find the averages. You'll also notice that we included arguments for filtering with 1B and 500M, based on the install numbers for Google Play. We need to use either 1B or 500M, similarly to the logic for iOS filters, ie. we can't use both at the same time (using the 500M+ filter will exclude apps with 1B+ installs anyway). The same applies to the sum and the avg parameters.

In [128]:
def android_number_installs(sum_installs=False, avg_installs=False, 
                           filter1B=False, filter500M=False):
    
    android_number_installs_category = {}

    for category in android_category:
        sum_installs_category = 0
        sum_apps_category = 0 # number of apps 
            
        for app in android_english_free:
            category_app = app[1] # Category
            installs = app[5] # Installs
            installs = installs.replace("+", "")
            installs = installs.replace(",", "")
            installs = int(installs)
            
            if category_app != category:
                continue
            if installs == 0:
                continue # excluding apps with 0 installs

            if filter1B and not filter500M and installs >= 1000000000:
                continue # skip this app, it's too popular
            elif filter500M and installs >= 500000000:
                continue # skip this app, also too popular
                
            sum_installs_category += installs
            sum_apps_category += 1

        if avg_installs and not sum_installs:
            avg_n_installs = round(sum_installs_category / sum_apps_category)
            android_number_installs_category[category] = avg_n_installs
            
        elif sum_installs and not avg_installs:
            android_number_installs_category[category] = sum_installs_category
    
    return display_sorted_table(android_number_installs_category)

android_number_installs(avg_installs=True)

COMMUNICATION : 38456119
VIDEO_PLAYERS : 24727872
SOCIAL : 23352604
PHOTOGRAPHY : 17840110
PRODUCTIVITY : 16787331
GAME : 15588016
TRAVEL_AND_LOCAL : 13984078
ENTERTAINMENT : 11640706
TOOLS : 10801391
NEWS_AND_MAGAZINES : 9587839
BOOKS_AND_REFERENCE : 8767812
SHOPPING : 7036877
PERSONALIZATION : 5201483
WEATHER : 5074486
HEALTH_AND_FITNESS : 4188822
MAPS_AND_NAVIGATION : 4056942
FAMILY : 3697848
SPORTS : 3638640
ART_AND_DESIGN : 2021805
FOOD_AND_DRINK : 1924898
EDUCATION : 1833495
BUSINESS : 1716508
LIFESTYLE : 1437816
FINANCE : 1387692
HOUSE_AND_HOME : 1331541
DATING : 854029
COMICS : 817657
AUTO_AND_VEHICLES : 647318
LIBRARIES_AND_DEMO : 638504
PARENTING : 542604
BEAUTY : 513152
EVENTS : 253542
MEDICAL : 120551


On average, Communication apps have the most installs: 38,456,119. This category also has the highest number of reviews, like demonstrated previously. The installs number is heavily skewed up by a few apps with over 1B installs (WhatsApp, Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 500M and 100M installs:

In [143]:
for app in android_english_free:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

The same can probably be said for the other categories as well, but we won't delve into it at the moment because we already covered this with the number of user reviews. We can also build a formula specifically for the installs, or adjust some of the existing ones. For now, what's important to note is that compared to the rankings in the number of reviews section, the top 10 categories are pretty much the same, which ultimately doesn't affect the recommendations that much.

## Conclusions

In this project, we analyzed data about mobile apps from the Google Play and the iOS App Store, with the goal of recommending an app profile that is likely to attract users and be profitable for the company in both markets. The company makes free apps and the main source of revenue is in-app adds, so it needs to build an app that has the potential to attract a lot of users, who will use the app for a longer duration, and is ideally suitable for showing relevant ads.

The project included the following steps:
1. Opening and exploring the data
2. Cleaning up the data by removing duplicates and incorrect data
3. Filtering only the free English apps
4. Analyzed the most common and most popular apps on each market.

We've used the number of reviews as the main criteria for popularity, but we can broaden this analysis by including data for number of installs, duration of usage of the apps, interaction with the ads, and the proportion of positive vs negative ratings. Some of these are not available in the dataset, which means we'd need to collect additional data. We also needed to consider if and how to exclude outliers, ie. apps with relatively high number of user reviews that skew the averages. 

Based on the recommendations for both datasets, we can conclude that the genres/categories that seem reasonable are the ones where users to go search for things that can technically be bought, and find avertisers that will promote directly related products and services that won't overwhelm, distract, or push away the users. Good contenders are Reference, Business, Productivity, Travel, Health & Fitness and Lifestyle. These genres/categories have variety between the apps within, which eases the oversaturation problem and allows for creativity and differentiation, and even combine multiple genres that can help broaden the scope or functionalities.

However, the popularity of the apps is not the only things that matters. We also need to consider how the apps work with ads, what the state of the market is, and what capabilities and resources the company has available. We provided reasoning for specific genres/categories, but here we're focusing on the general conclusions.

We need to ensure that the ads will be seen and users will spend time using the apps, so wallpaper customization that get opened once or security apps that run in the background are not a great idea. Next, we ideally want to avoid genres/categories with oversaturation and very strong competition, like Games and Entertainment, especially if big giants like Facebook and Netflix dominate the market - unless we have a very creative idea that is worth testing and the human resources to make it happen. Given the markets' saturation, a practical app might be more succesful than a for-fun app. We also want to avoid additional expenses through licences or serves, or niche categories that would likely require specific domain knowledge the company might not have, like Sports, Finance and Medical.  We're also limited in creating apps for Food & Drinks delivery or Shopping without physically offering those services and products, unless we develop it for a business partner, or create an app with ideas, recepies, recommendations, and networking with other users, where we can connect them with the products and services through the ads.

## Extra

### Check if app is in dataset

If you'd like to check whether a specific app is in the Google Play dataset (or iOS dataset), use the below code and change the name of the `check_this` variable. This can also be used to check if the dataset has any mispelled names. There are two counters, one counts how much times the app is mentioned on its own (eg "Google Maps"). The other counts how much times the chosen name is mentioned within the app name (eg "Maps" in "Google Maps"), which is accompanied by a list of the mentions. We'll use the original datasets, before removing any duplicates and rows with innacurate data, ie. `android_apps`.

The example with Instagram below returns 4 rows with just "Instagram" in the name and 16 mentions as a substring. By looking at the mentions list or filtering the dataset in Excel, you'll notice that there are indeed other apps that contain "Instagram" in their name, that aren't necessarily referring to the app itself.

In [144]:
check_this = "Facebook"

number_times_app_found = 0 # just the app by itself
mentions_within_name = 0 # substring
mentions_within_name_list = []

for app in android_apps:
    name = app[0]
    if check_this == name:
        number_times_app_found += 1
    
    if check_this in name:
        mentions_within_name += 1
        mentions_within_name_list.append(name)
    
print("Number of times the app is found:", number_times_app_found)
print("Number of mentions within the name:", mentions_within_name, "\n")
print(mentions_within_name_list)

Number of times the app is found: 2
Number of mentions within the name: 33 

['Facebook Pages Manager', 'Facebook Ads Manager', 'Facebook', 'Facebook Lite', 'Who Viewed My Facebook Profile - Stalkers Visitors', 'Facebook Local', 'HTC Social Plugin - Facebook', 'Facebook', 'Facebook Lite', 'Video Downloader for Facebook', 'Dp for Facebook', 'EZ Video Download for Facebook', 'Facebook Face to Face Events', 'Facebook Creator', 'Swift for Facebook Lite', 'Friendly for Facebook', 'Facebook Pages Manager', 'Faster for Facebook Lite', 'Facebook Ads Manager', 'Puffin for Facebook', 'Lite for Facebook Messenger', 'Profile Tracker - Who Viewed My Facebook Profile', 'Pink Color for Facebook', 'Who Viewed My Facebook Profile - Stalkers Visitors', 'Phoenix - Facebook & Messenger', 'Faster Social for Facebook', 'Videos downloader for Facebook:fast fb video saver', 'Stickers for Facebook', 'Lite Messenger for Facebook Lite', 'Download Facebook Photo Albums', 'Mini for Facebook lite', 'IDM for Faceboo

In [145]:
# Extra practice code

example_app_names = ["Instagram", "Facebook"]

print("Instagram" in example_app_names)
print("Twitter" in example_app_names)
print(232 in example_app_names)
print("Facebook" in example_app_names)

True
False
False
True


### Identifying free apps through the "Type" column for Google Play

We already checked for free apps using the "Price" column, but let's check with the "Type" (Free or Paid) column too, to confirm that it correctly identifies the same number of free apps.

In [146]:
android_type_free = []
android_type_paid = []
android_type_remaining = []

for app in android_english:
    name = app[0]
    price = app[7]
    free_vs_paid = app[6] # Type
    
    if free_vs_paid == "Free":
        android_type_free.append(app)
    elif free_vs_paid == "Paid":
        android_type_paid.append(app)
    else:
        android_type_remaining.append(app)

print("Free:", len(android_type_free))
print("Paid:", len(android_type_paid))
print("Other:", len(android_type_remaining))
print(android_type_remaining[0][0])
print("price:", android_type_remaining[0][7])

Free: 8863
Paid: 750
Other: 1
Command & Conquer: Rivals
price: 0


This column identifies 8,863 free apps and 1 more that is not identified as either free or paid, which is "Command & Conquer: Rivals" with a price of 0. When we add it to the "Free" apps, the number turns out to be 8,864 apps, which is the same as the total number identified through the "Price" column.

### Including thousand separators with f-strings

We format the number 1000000000 to inlude thousand separators using multiple methods, which converts it into a string.

In [147]:
num = 1000000000
print(num, type(num))

num_fstring = f"{num:,d}"
print(num_fstring, type(num_fstring))

res = '{:,}'.format(1000000000)
print(res, type(res))

from decimal import Decimal
number = Decimal("123456789")
print(number, type(number))
formatted_number = format(number, ',')
print(formatted_number, type(formatted_number))

1000000000 <class 'int'>
1,000,000,000 <class 'str'>
1,000,000,000 <class 'str'>
123456789 <class 'decimal.Decimal'>
123,456,789 <class 'str'>


### UnicodeDecodeError when opening the datasets
If you run into an error named `UnicodeDecodeError`, add `encoding="utf8"` to the `open()` function. For example `open('AppleStore.csv', encoding="utf8")`.

## Sources

1. Google Play dataset (incl. documentation):
    - L. Gupta, "Google Play Store Apps", February 2019. [Online]. Available at: https://www.kaggle.com/datasets/lava18/google-play-store-apps
2. Google Play direct CSV download: 
    - https://dq-content.s3.amazonaws.com/350/googleplaystore.csv
3. Google Play discussions section: 
    - https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion
4. Google Play row 10472 error discussion: 
    - https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion/164101
5. iOS dataset (incl. documentation):
    - R. Perumal, "Mobile App Store (7200 apps)", July 2017. [Online]. Available at: https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps
6. iOS direct CSV download: 
    - https://dq-content.s3.amazonaws.com/350/AppleStore.csv
7. iOS discussions section: 
    - https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps/discussion
8. Statista:
    -  L. Ceci, "Number of apps available in leading app stores as of August 2024", August 2024. [Online]. Available at: https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/
9. Photo from Dursikshya blog: 
    - https://dursikshya.edu.np/details/mobile-app-architecture-and-how-to-start-building-one
10. Family/Kids category on Google Play: 
    - https://play.google.com/store/apps/category/FAMILY?hl=en
11. iOS genres explanation: 
    - https://developer.apple.com/app-store/categories/